In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from env import user, password, host
from scipy.stats import levene, ttest_ind
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
import math
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import statsmodels.api as sm
import wrangle_campus as wc
import warnings
warnings.filterwarnings("ignore")

In [2]:
df22 = pd.read_csv('CAMPUS_summary_22.csv')
df21 = pd.read_csv('CAMPUS_summary_21.csv')
df20 = pd.read_csv('CAMPUS_summary_20.csv')
df19 = pd.read_csv('CAMPUS_summary_19.csv')
df18 = pd.read_csv('CAMPUS_summary_18.csv')

In [3]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,533
1,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A02,CAMPUS DISCIPLINE POPULATION,27
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,40
3,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,B-DISCIPLINE DATA TRENDS,B13,STUDENTS SUSPENDED OUT OF SCHOOL,27
4,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,F-OUT OF SCHOOL SUSPENSIONS,C24,BLACK OR AFRICAN AMERICAN,-999
...,...,...,...,...,...,...,...,...,...,...
323988,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,V-AT RISK IN SCHOOL SUS.,F19,NON AT RISK IN SCHOOL SUSPENSIONS,-999
323989,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,W-REASON INCIDENT COUNTS,G21,21-VIOLATED LOCAL CODE OF CONDUCT,-999
323990,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,W-REASON INCIDENT COUNTS,G61,61-BULLYING  TEC 37.0052(B),-999
323991,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,X-DISCIPLINE ACTION COUNTS,H05,05-OUT-OF-SCHOOL SUSPENSION,-999


In [4]:
df22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323993 entries, 0 to 323992
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   agg_level        323993 non-null  object
 1   campus_number    323993 non-null  int64 
 2   region           323993 non-null  int64 
 3   dist_name_num    323993 non-null  object
 4   charter_status   323993 non-null  object
 5   campus_name_num  323993 non-null  object
 6   section          323993 non-null  object
 7   heading          323993 non-null  object
 8   heading_name     323993 non-null  object
 9   student_count    323993 non-null  object
dtypes: int64(2), object(8)
memory usage: 24.7+ MB


In [5]:
print(wc.prep22(df22))

None


In [6]:
wc.prep22(df22)

AttributeError: 'DataFrame' object has no attribute 'charter_status'

In [ ]:
df22.head()